## Test for interventions

In [4]:
from snowplow_signals import Signals
from dotenv import load_dotenv
import os

load_dotenv()

sp_signals = Signals(
    api_url=os.environ["SP_API_URL"],
    api_key=os.environ["SP_API_KEY"],
    api_key_id=os.environ["SP_API_KEY_ID"],
    org_id=os.environ["SP_ORG_ID"],
)

### Define a new attribute

This block creates a single feature definition including the logic how it should be calculated (it's filters and aggregation).

The feature calculates the number of add to cart ecommerce events.

In [65]:
from snowplow_signals import (
    Attribute,
    Event,
    Criterion,
    Criteria,
    EventProperty,
    EntityProperty,
)

count_product_views = Attribute(
    name="count_product_views",
    type="int32",
    events=[Event(name="snowplow_ecommerce_action")],
    criteria=Criteria(
        all=[
            Criterion.eq(
                property=EventProperty(
                    vendor="com.snowplowanalytics.snowplow.ecommerce",
                    name="snowplow_ecommerce_action",
                    major_version=1,
                    path="type",
                ),
                value="product_view",
            )
        ]
    ),
    aggregation="counter",
)

count_add_to_cart = Attribute(
    name="count_add_to_cart",
    type="int32",
    events=[Event(name="snowplow_ecommerce_action")],
    criteria=Criteria(
        all=[
            Criterion.eq(
                property=EventProperty(
                    vendor="com.snowplowanalytics.snowplow.ecommerce",
                    name="snowplow_ecommerce_action",
                    major_version=1,
                    path="type",
                ),
                value="add_to_cart",
            )
        ]
    ),
    aggregation="counter",
)

total_cart_value = Attribute(
    name="total_cart_value",
    type="int32",
    events=[Event(name="snowplow_ecommerce_action")],
    criteria=Criteria(
        all=[
            Criterion.eq(
                property=EventProperty(
                    vendor="com.snowplowanalytics.snowplow.ecommerce",
                    name="snowplow_ecommerce_action",
                    major_version=1,
                    path="type",
                ),
                value="add_to_cart",
            )
        ]
    ),
    property=EntityProperty(
        vendor="com.snowplowanalytics.snowplow.ecommerce",
        name="product",
        major_version=1,
        path="price",
    ),
    aggregation="sum",
)


### Wrapping the attribute in a group

All features need to be included in feature views that can be considered as "tables" of features.

Attribute groups are immutable and versioned.

In [68]:
from snowplow_signals import StreamView, domain_userid

view = StreamView(
    name="ecom_attributes",
    version=1,
    entity=domain_userid,
    attributes=[
        count_product_views,
        count_add_to_cart,
        total_cart_value,
    ],
    owner="user@company.com",
)

### Testing the view

Execute the feature view on the last one hour of data from the atomic events table to verify that it works correctly.

In [69]:
sp_signals.test(view=view)

,domain_userid,count_product_views,count_add_to_cart,total_cart_value
0,66e7d802-2e75-40ee-8479-74caaaefaf38,0,0,0.0
1,839743b9-e629-49af-9071-e34430b9bcac,0,0,0.0
2,9b1de135-cf1f-46cb-a071-33fde0bb5400,0,0,0.0
3,76401be1-70b9-4ba9-a2c2-a619a73cfbbe,0,0,0.0
4,1b3870ea-5654-46d9-beb2-6e7c61eb1f56,0,0,0.0
5,286e398f-7b85-4dad-8451-d49155735b02,0,0,0.0
6,1f218973-57b4-40a2-a9fe-689490963f27,0,0,0.0
7,8b1a4ae7-2246-4275-9a9c-19d1012e1c73,0,0,0.0
8,8412a451-8c85-4645-9fb6-fc73eaa2f6b1,0,0,0.0
9,7f245827-9062-4d77-862a-a1b50653413b,0,0,0.0


# Wrap the attribute group in a service

In [70]:
from snowplow_signals import Service

service = Service(
    name="ecom_attributes",
    views=[view],
    owner="user@company.com",
)

# Publish the attribute group and service

Pulbish the attribute group to start processing it.

In [71]:
sp_signals.apply([view, service])

[View(name='ecom_attributes', version=1, entity=Entity(name='domain_userid', description=None, key=None, tags=None, owner=None, ttl=None), ttl=None, batch_source=None, online=True, offline=False, description=None, tags=None, owner='user@company.com', fields=[], attributes=[AttributeInput(name='count_product_views', description=None, type='int32', tags=None, events=[Event(name='snowplow_ecommerce_action', vendor=None, version=None)], aggregation='counter', property_syntax='snowflake', property=None, criteria=Criteria(all=[SignalsApiModelsViewCriterionCriterion(property_syntax='snowflake', property='unstruct_event_com_snowplowanalytics_snowplow_ecommerce_snowplow_ecommerce_action_1:type', operator='=', value='product_view')], any=None), period=None, default_value=None), AttributeInput(name='count_add_to_cart', description=None, type='int32', tags=None, events=[Event(name='snowplow_ecommerce_action', vendor=None, version=None)], aggregation='counter', property_syntax='snowflake', property

# Retrieve attributes for the service

In [80]:
service.get_attributes(
    signals=sp_signals,
    entity="domain_userid",
    identifier="be240d67-f4bf-4f66-9b19-0338a31c1c17",
)

{'domain_userid': 'be240d67-f4bf-4f66-9b19-0338a31c1c17',
 'count_add_to_cart': 1,
 'count_product_views': 3,
 'total_cart_value': 12.99}

In [74]:
from snowplow_signals import (
    RuleIntervention,
    InterventionCriterion,
)

cart_abandonment_intervention = RuleIntervention(
    name="cart_abandonment",
    description="Show banner to users who have added at least one item to cart.",
    method="remote_agent",
    criteria=InterventionCriterion(
        attribute="ecom_attributes:count_add_to_cart",
        operator=">",
        value=0,
    ),
    owner="user@company.com",
)

discount_intervention = RuleIntervention(
    name="discount",
    description="Show banner to users who have viewed more than 3 products.",
    method="remote_agent",
    criteria=InterventionCriterion(
        attribute="ecom_attributes:count_product_views",
        operator=">",
        value=3,
    ),
    owner="user@company.com",
)

free_shipping_intervention = RuleIntervention(
    name="free_shipping",
    description="Show banner to users who have spent at least $50.",
    method="remote_agent",
    criteria=InterventionCriterion(
        attribute="ecom_attributes:total_cart_value",
        operator=">",
        value=100,
    ),
    owner="user@company.com",
)


### Apply the intervention

In [75]:
sp_signals.apply(
    [
        cart_abandonment_intervention,
        discount_intervention,
        free_shipping_intervention,
    ]
)


[RuleIntervention(name='cart_abandonment', version=1, method='remote_agent', target_agents=None, script_uri=None, context={}, description='Show banner to users who have added at least one item to cart.', tags=None, owner='user@company.com', criteria=SignalsApiModelsInterventionCriterionCriterion(attribute='ecom_attributes:count_add_to_cart', operator='>', value=0)),
 RuleIntervention(name='discount', version=1, method='remote_agent', target_agents=None, script_uri=None, context={}, description='Show banner to users who have viewed more than 3 products.', tags=None, owner='user@company.com', criteria=SignalsApiModelsInterventionCriterionCriterion(attribute='ecom_attributes:count_product_views', operator='>', value=3)),
 RuleIntervention(name='free_shipping', version=1, method='remote_agent', target_agents=None, script_uri=None, context={}, description='Show banner to users who have spent at least $50.', tags=None, owner='user@company.com', criteria=SignalsApiModelsInterventionCriterionC